In [7]:
import scipy.io
import cv2
import matlab.engine
import numpy as np
import os
import pandas as pd
from PIL import Image
from skimage import morphology
from skimage import io

from config import considered_obj
import skimage

In [2]:
path = os.path.join('ADE20K_2016_07_26', 'images', 'training', 'b', 'bedroom')

img_path = {} # {'img_name': 'path'}
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.jpg' in file:
            img_path[file.split('.jpg')[0]] = os.path.join(r, file)

In [3]:
eng = matlab.engine.start_matlab()

# Load segments

In [4]:
# img_path = {'ADE_train_00003679': 'ADE20K_2016_07_26/images/training/b/bedroom/ADE_train_00003679.jpg'}

In [5]:
for img_name in img_path:

    Om, Oi, Pm, Pi, objects, parts = eng.loadAde20K(img_path[img_name], nargout=6)

    np.save('loaded_segmentation/' + img_name +'_om.npy', np.array(Om))
    np.save('loaded_segmentation/' + img_name + '_oi.npy', np.array(Oi))
    np.save('loaded_segmentation/'+ img_name + '_pm.npy', np.array(Pm))
    np.save('loaded_segmentation/' + img_name + '_pi.npy', np.array(Pi))

    df_objects = pd.DataFrame.from_dict(objects)
    df_parts = pd.DataFrame.from_dict(parts)

    df_objects.to_csv('loaded_segmentation/' + img_name + '_objects.csv')
    df_parts.to_csv('loaded_segmentation/' + img_name + '_parts.csv')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-0fecc8e6b531>", line 8, in <module>
    np.save('loaded_segmentation/' + img_name + '_pi.npy', np.array(Pi))
  File "/Users/sky/Documents/GitRepositories/partialconv/env/bin/../lib/python3.5/_collections_abc.py", line 815, in __iter__
    v = self[i]
  File "/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/matlab/_internal/mlarray_sequence.py", line 79, in __getitem__
    def __getitem__(self, index):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2039, in showtraceback
    stb = value._render_tra

KeyboardInterrupt: 

# Extract segments

In [8]:
for img_name in img_path:

    original_img = Image.open(img_path[img_name])
    original_img = np.array(original_img)

    segment_class = np.load('loaded_segmentation/' + img_name + '_oi.npy')
    segment_class_name = pd.read_csv('loaded_segmentation/' + img_name + '_objects.csv')

    segmented_imgs = list()

    for i in np.unique(segment_class):
        if (segment_class_name['class'].iloc[int(i) - 1] in considered_obj) and (int(i) != 0):
            prediction_mask = (segment_class == i)
            prediction_box = (segment_class == i)
            prediction_box = prediction_box.astype(np.uint8)
            prediction_box = prediction_box*255

            cropped_object = original_img * np.dstack((prediction_mask,) * 3)
            square = morphology.disk(5)

            temp = morphology.dilation(prediction_mask, square)
            temp = morphology.binary_dilation(temp, square)
            temp = morphology.binary_dilation(temp, square)
            temp = morphology.binary_dilation(temp, square)
            
            path_to_save = os.path.join('extracted_segments', img_name + str(i) + '.png')
            segmented_imgs.append(path_to_save)
            io.imsave(path_to_save, skimage.img_as_float32(temp))
            
            negative_mask = (temp != True)
            eroding_countour = negative_mask * prediction_mask
            eroding_countour_img = np.dstack((eroding_countour, ) * 3)
            cropped_object[eroding_countour_img] = 0
            png_transparancy_mask = np.uint8(prediction_mask * 255)
            image_shape = cropped_object.shape
            png_array = np.zeros(shape=[image_shape[0], image_shape[1], 4], dtype=np.uint8)
            png_array[:, :, :3] = cropped_object
            png_array[:, :, 3] = png_transparancy_mask

#             path_to_save = os.path.join('extracted_segments', img_name + str(i) + '.png')
#             segmented_imgs.append(path_to_save)
#             io.imsave(path_to_save, png_array)

    network_input = Image.open(img_path[img_name])
    network_input = network_input.convert('RGBA')

    network_input_data = network_input.load()

    for segmented_img in segmented_imgs:
        segmented = Image.open(segmented_img)
        segmented = segmented.convert('RGB')
        segmented_data = segmented.load()

        for y in range(segmented.size[1]):
            for x in range(segmented.size[0]):
                if segmented_data[x, y] != (0, 0, 0):
                    network_input_data[x, y] = (255, 255, 255, 255)
                    segmented_data[x, y] = (0, 0, 0)
                else:
                    segmented_data[x, y] = (255, 255, 255)
        segmented.save('masks/' + segmented_img.split('/')[1].split('.png')[0] + '.png')

    network_input.save('network_input/' + img_name + '.png')

/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000019616.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000046930.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000046931.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000381718.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-

/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000420421.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000041411.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000041412.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000041413.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)


/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000377019.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000331221.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)


/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000051121.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000051125.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000051127.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000051131.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-

/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000389414.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000389417.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000389418.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000389420.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-

...

/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000352616.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000032927.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_000003717.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)


/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000037118.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000037121.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000037125.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000358014.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-

/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000040614.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_000002675.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000026712.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000047015.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-p

/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000385117.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_000037728.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/sky/Documents/GitRepositories/partialconv/env/lib/python3.5/site-packages/skimage/io/_io.py:141: UserWarning: extracted_segments/ADE_train_0000348814.0.png is a low contrast image
  warn('%s is a low contrast image' % fname)


FileNotFoundError: [Errno 2] No such file or directory: 'loaded_segmentation/ADE_train_00003908_oi.npy'